In [1]:
#START: take query
# CONDITION: check sentiment

# A: negative
    # run diagnosis
    # respond

# B: positive
    # respond

#END

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph import START, END, StateGraph
from dotenv import load_dotenv
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
# from langchain_core.output_parsers import JsonOutputParser

In [3]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [4]:
class SentimentSchema(BaseModel):
    sentiment: Literal["Positive", "Negative"] = Field(description="Sentiment of the review")

class DiagnosisSchema(BaseModel):
    tone: Literal["angry", "calm", "disappointed"] = Field(description="Tone of the review")
    issue: str = Field(description="The category of issue being faced")

In [5]:
structured_sentiment = model.with_structured_output(SentimentSchema)
structured_diagnosis = model.with_structured_output(DiagnosisSchema)

NotImplementedError: Pydantic schema is not supported for function calling

In [ ]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["Positive", "Negative"]
    response: str
    diagnosis: dict

In [ ]:
def checkReview(state: ReviewState) -> dict:
    answer = structured_sentiment.invoke(f"Check the sentiment of this review: {state['review']}" ).sentiment.strip()
    return {'sentiment': answer}

# =====================Router==================================
def classifyReview(state: ReviewState) -> str:
    if state["sentiment"] == "Positive":
        return "Positive_Review"
    else:
        return "runDiagnosis"
# ============================================================


def Positive_Review() -> dict:
        answer = model.invoke("Write a thank you message").content
        return {'response': answer}

def runDiagnosis(state: ReviewState) -> dict:
    answer =  structured_diagnosis.invoke(f"tell the following things about this review: tone of the message, issue faced {state['review']}")
    return {'diagnosis':answer.model_dump() }

def Negative_Review(state: ReviewState) ->dict:
    
    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
    The user had a '{diagnosis['issue']}' issue and sounded '{diagnosis['tone']}',
    Write an empathetic, helpful resolution message.
    """
    response = model.invoke(prompt).content

    return {'response': response}

In [ ]:
graph = StateGraph(ReviewState)

graph.add_node("checkReview", checkReview)
graph.add_node("Positive_Review", Positive_Review)
graph.add_node("runDiagnosis", runDiagnosis)
graph.add_node("Negative_Review", Negative_Review)

graph.add_edge(START, "checkReview")

graph.add_conditional_edges("checkReview", classifyReview)

graph.add_edge("Positive_Review", END)
graph.add_edge("runDiagnosis", "Negative_Review")
graph.add_edge("Negative_Review", END)

workflow = graph.compile()

workflow